In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

#Importing NFL stats dataset
file_path = '/content/drive/My Drive/Sports Modeling/nfl/full_nfl.csv'
data = pd.read_csv(file_path)

<ipython-input-2-44bd5e901970>:5: DtypeWarning: Columns (10,442,477,480,488,500,505) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path)


In [ ]:
data[['home_name', 'away_name','home_record_wins', 'away_record_wins', 'home_record_losses', 'away_record_losses']].count()

,0
home_name,6053
away_name,6053
home_record_wins,824
away_record_wins,824
home_record_losses,824
away_record_losses,824


In [ ]:
# Count of home_points == away_points (draws)
draws_count = (data['home_points'] == data['away_points']).sum()

# Count of home_points > away_points (home team wins)
home_wins_count = (data['home_points'] > data['away_points']).sum()

# Count of home_points < away_points (away team wins)
away_wins_count = (data['home_points'] < data['away_points']).sum()

# Display the counts
print(f"Draws: {draws_count}")
print(f"Home Wins: {home_wins_count}")
print(f"Away Wins: {away_wins_count}")

Draws: 14
Home Wins: 3404
Away Wins: 2635


In [ ]:
# Count the number of columns
num_columns = data.shape[1]

# Display the count
print(f"Number of columns in the dataset: {num_columns}")


Number of columns in the dataset: 679


In [ ]:
  # Get the count of null values in each column
null_counts = data.isnull().sum()

# Filter the columns with null values
null_counts = null_counts[null_counts > 0]

# Display the columns with their respective null value counts
print(null_counts)

# Optionally, save the null counts to a CSV file for further analysis
null_counts.to_csv('null_value_counts.csv', header=['null_counts'])

# If you want to download the file, use the following code
from google.colab import files
files.download('null_value_counts.csv')

venue_state                                  44
venue_zip                                    19
home_rushing_totals_longest_touchdown       300
home_receiving_totals_longest_touchdown     142
home_punts_totals_longest                    40
                                           ... 
BarstoolUnderPayout                        5767
BarstoolHomeTeamTotal                      6053
BarstoolAwayTeamTotal                      6053
BarstoolHomeTeamTotalPayout                6053
BarstoolAwayTeamTotalPayout                6053
Length: 328, dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Calculate the threshold for dropping columns
threshold = len(data) / 2  # 50% threshold

# Drop columns with more than 50% null values
data = data.dropna(thresh=threshold, axis=1)

In [ ]:
# Filter columns with dtype other than numerical (int64, float64)
non_numeric_columns = data.select_dtypes(exclude=['int64', 'float64']).columns

# Find non-numeric columns with null values and their respective null counts
null_counts = data[non_numeric_columns].isnull().sum()

# Filter to only include columns that have null values
columns_with_nulls = null_counts[null_counts > 0]

# Print the columns with their null counts
print(columns_with_nulls)

venue_state       44
venue_zip         19
venue_country    448
dtype: int64


In [ ]:
# Select columns with dtypes other than numeric (int64, float64)
non_numeric_columns = data.select_dtypes(exclude=['int64', 'float64']).columns

# Print the non-numeric columns
print(non_numeric_columns)

Index(['scheduled', 'season_name', 'venue_name', 'venue_city', 'venue_state',
       'venue_zip', 'venue_address', 'venue_surface', 'venue_roof_type',
       'home_name', 'home_market', 'home_alias', 'away_name', 'away_market',
       'away_alias', 'home_possession_time', 'away_possession_time',
       'venue_country', 'time'],
      dtype='object')


In [ ]:
# List of columns to remove which has least relevance with prediction variable
columns_to_remove = [
    'scheduled', 'season_name', 'venue_state', 'venue_zip', 'venue_address',
    'home_market', 'home_alias', 'away_market', 'away_alias', 'venue_country', 'time'
]

# Drop the specified columns
data = data.drop(columns=columns_to_remove)

In [ ]:
# Get the count of null values in each column
null_counts = data.isnull().sum()

# Filter the columns with null values
null_columns = null_counts[null_counts > 0].index.tolist()

# Display the columns with their respective null value counts
print(null_columns)

['home_rushing_totals_longest_touchdown', 'home_receiving_totals_longest_touchdown', 'home_punts_totals_longest', 'home_punt_returns_totals_longest', 'home_punt_returns_totals_longest_touchdown', 'home_passing_totals_longest_touchdown', 'home_kick_returns_totals_longest', 'home_kick_returns_totals_longest_touchdown', 'home_field_goals_totals_longest', 'home_extra_points_conversions_totals_turnover_successes', 'away_rushing_totals_longest_touchdown', 'away_receiving_totals_longest_touchdown', 'away_punts_totals_longest', 'away_punt_returns_totals_longest', 'away_punt_returns_totals_longest_touchdown', 'away_passing_totals_longest_touchdown', 'away_kick_returns_totals_longest', 'away_kick_returns_totals_longest_touchdown', 'away_field_goals_totals_longest', 'away_extra_points_conversions_totals_turnover_successes', 'venue_location_lat', 'venue_location_lng', 'home_int_returns_totals_longest', 'home_int_returns_totals_longest_touchdown', 'away_kickoffs_totals_onside_attempts', 'away_kicko

### Finding Skewed and Symmetric columns

In [ ]:
# Identify columns with null values
null_columns = data.columns[data.isnull().any()].tolist()

# Calculate skewness for numeric columns
skewness = data.select_dtypes(include=['int64', 'float64']).skew()

# Define threshold for symmetric columns
threshold = 0.5

# Identify symmetric and skewed columns
symmetric_columns = skewness[abs(skewness) < threshold].index.tolist()
skewed_columns = skewness[abs(skewness) >= threshold].index.tolist()

# Find columns that are symmetric with null values
symmetric_with_nulls = [col for col in symmetric_columns if col in null_columns]

# Find columns that are skewed with null values
skewed_with_nulls = [col for col in skewed_columns if col in null_columns]

# Print the results
print("Columns with null values that are symmetric:")
print(symmetric_with_nulls)

print("\nColumns with null values that are skewed:")
print(skewed_with_nulls)

Columns with null values that are symmetric:
['home_punts_totals_longest', 'home_extra_points_conversions_totals_turnover_successes', 'away_punts_totals_longest', 'away_extra_points_conversions_totals_turnover_successes', 'home_passing_totals_blitzes', 'away_passing_totals_blitzes', 'home_passing_totals_pocket_time', 'home_punts_totals_hang_time', 'away_punts_totals_hang_time', 'away_passing_totals_pocket_time']

Columns with null values that are skewed:
['home_rushing_totals_longest_touchdown', 'home_receiving_totals_longest_touchdown', 'home_punt_returns_totals_longest', 'home_punt_returns_totals_longest_touchdown', 'home_passing_totals_longest_touchdown', 'home_kick_returns_totals_longest', 'home_kick_returns_totals_longest_touchdown', 'home_field_goals_totals_longest', 'away_rushing_totals_longest_touchdown', 'away_receiving_totals_longest_touchdown', 'away_punt_returns_totals_longest', 'away_punt_returns_totals_longest_touchdown', 'away_passing_totals_longest_touchdown', 'away_kic

### Imputing null values

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np

# Filter out columns with insufficient data for imputation
symmetric_with_nulls = [col for col in symmetric_with_nulls if data[col].notnull().any()]
skewed_with_nulls = [col for col in skewed_with_nulls if data[col].notnull().any()]

# Calculate overall column means and medians for fallback
overall_means = data[symmetric_with_nulls].mean()
overall_medians = data[skewed_with_nulls].median()

# Define the imputation function
def impute_group(group, symmetric_cols, skewed_cols, overall_means, overall_medians):
    # Impute symmetric columns
    for col in symmetric_cols:
        if group[col].isnull().any():
            if group[col].notnull().sum() > 0:  # Use group-specific mean if possible
                mean_imputer = SimpleImputer(strategy='mean')
                group[col] = mean_imputer.fit_transform(group[[col]]).ravel()
            else:
                group[col] = group[col].fillna(overall_means[col])  # Fallback to overall mean

    # Impute skewed columns
    for col in skewed_cols:
        if group[col].isnull().any():
            if group[col].notnull().sum() > 0:  # Use group-specific median if possible
                median_imputer = SimpleImputer(strategy='median')
                group[col] = median_imputer.fit_transform(group[[col]]).ravel()
            else:
                group[col] = group[col].fillna(overall_medians[col])  # Fallback to overall median

    return group

# Apply the imputation based on 'home_name' and 'away_name'
try:
    data_imputed = data.groupby(['home_name', 'away_name'], group_keys=False).apply(
        lambda group: impute_group(group, symmetric_with_nulls, skewed_with_nulls, overall_means, overall_medians)
    ).reset_index(drop=True)
except ValueError as e:
    print(f"An error occurred during imputation: {e}")

In [ ]:
# Save the imputed DataFrame to a CSV file
data_imputed.to_csv('imputed_data.csv', index=False)